In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests # requests for the report website
import json
from sec_edgar_api import EdgarClient
from bs4 import BeautifulSoup # Import BeautifulSoup
import re # import re module for REGEXes
import time
import os
import codecs
import glob

In [2]:
# 读取Bloomberg overall数据
Bloom=pd.read_excel('ESG data/Overall Score Industries.xlsx')
Bloom['Ticker'] = Bloom['Ticker'].str.replace(r' \b\w{2} Equity\b', '', regex=True)
display(Bloom)

,NaN,Ticker,Bi074,Country,2022,2021,2020,2019,2018,2017,2016,2015
0,ACM Research Inc,ACMR,Semiconductors,United States,1.47,1.43,NaN,NaN,NaN,NaN,NaN,NaN
1,AXT Inc,AXTI,Semiconductors,United States,1.37,1.42,NaN,NaN,NaN,NaN,NaN,NaN
2,Advanced Micro Devices Inc,AMD,Semiconductors,United States,6.62,6.77,6.72,6.56,6.07,6.09,5.92,5.25
3,Allegro MicroSystems Inc,ALGM,Semiconductors,United States,2.19,2.02,NaN,NaN,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
4,Alpha & Omega Semiconductor Ltd,AOSL,Semiconductors,United States,1.98,2.01,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
188,Western Digital Corp,WDC,"Technology Hardware, Equipment and Components",United States,4.61,5.18,4.65,3.18,3.24,2.03,1.89,1.79
189,Wireless Telecom Group Inc,WTT,"Technology Hardware, Equipment and Components",United States,1.42,1.48,NaN,NaN,NaN,NaN,NaN,NaN
190,Xerox Holdings Corp,XRX,"Technology Hardware, Equipment and Components",United States,4.28,5.49,5.39,4.78,3.98,3.95,3.75,3.56
191,Zebra Technologies Corp,ZBRA,"Technology Hardware, Equipment and Components",United States,3.65,3.98,4.15,1.98,1.8,1.88,1.85,1.33


In [3]:
def extract_details_from_filename(filename,path):
    filename = filename.replace(f'{path}/','')
    parts = filename.replace('.csv', '').split('-')
    CompanyTicker = parts[0]
    Year = int(float(parts[1]))  # Convert to float first, then to int to handle any decimal points
    return CompanyTicker, Year

In [4]:
# Folder路径
folder_path = 'New Report Data 2015-now'
# Use glob to get all .csv files in the folder
csv_files = glob.glob(f'{folder_path}/*.csv')
ESGOverallData = pd.DataFrame(columns=['CompanyTicker','Year','CSV.Length','ESGData','ESGLabel','BloombergOverall'])

# Main loop to process all CSV files
for filename in csv_files:
    try:
        # Extract company ticker and year from the filename
        CompanyTicker, Year = extract_details_from_filename(filename, folder_path)
        
        # Read the CSV file into a DataFrame
        ESGData = pd.read_csv(filename)
        # display(ESGData)
        # Initialize an empty list to store ESG scores for each row
        CSVRowESG = []
        RowLabel = []
        # Iterate over each row in the DataFrame
        for index, row in ESGData.iterrows():
            # Extract ESG scores and append to the list
            ESGN = [row['Env'], row['Soc'], row['Gov'], row['None']]
            CSVRowESG.append(ESGN)
            Max_value = max(ESGN)
            if Max_value == row['Env']:
                RowLabel.append('E')
            elif Max_value == row['Soc']:
                RowLabel.append('S')
            elif Max_value == row['Gov']:
                RowLabel.append('G')
            elif Max_value == row['None']:
                RowLabel.append('N')

        # Filter Bloomberg data for the current company ticker
        TickerBloom = Bloom[Bloom['Ticker'] == CompanyTicker]
        
        # Extract the overall ESG score for the current year
        TickerBloomOverall = TickerBloom[Year].values[0]
        
        # Print the overall ESG score
        #print(float(TickerBloomOverall))
        
        # Append the data to the ESGOverallData DataFrame
        ESGOverallData.loc[len(ESGOverallData)] = [CompanyTicker, Year,len(ESGData), CSVRowESG,RowLabel, TickerBloomOverall]
        
    except Exception as e:
        # Print error message if there is an issue reading the file
        print(f"{CompanyTicker}-{Year}读取文件时发生错误: {e}")
        # Set the overall ESG score to 0 in case of an error
        ESGOverallData.loc[len(ESGOverallData)] = [CompanyTicker, Year,len(ESGData), CSVRowESG,RowLabel, 0]

display(ESGOverallData)

CRUS-2024读取文件时发生错误: 2024
GPRO-2023读取文件时发生错误: 2023
FFIV-2023读取文件时发生错误: 2023
NEON-2023读取文件时发生错误: 2023
ANET-2023读取文件时发生错误: 2023
HLIT-2023读取文件时发生错误: 2023
AAPL-2023读取文件时发生错误: 2023
CRUS-2023读取文件时发生错误: 2023
MXL-2023读取文件时发生错误: 2023
SYNA-2023读取文件时发生错误: 2023
UI-2023读取文件时发生错误: 2023
COHR-2023读取文件时发生错误: 2023
CMBM-2023读取文件时发生错误: 2023
KLAC-2023读取文件时发生错误: 2023
HPE-2023读取文件时发生错误: 2023
COHU-2023读取文件时发生错误: 2023
OLED-2023读取文件时发生错误: 2023
MKSI-2023读取文件时发生错误: 2023
VYX-2023读取文件时发生错误: 2023
SWKS-2023读取文件时发生错误: 2023
AMBA-2023读取文件时发生错误: 2023
AAOI-2023读取文件时发生错误: 2023
LITE-2023读取文件时发生错误: 2023
NTAP-2023读取文件时发生错误: 2023
LRCX-2023读取文件时发生错误: 2023
AMBA-2024读取文件时发生错误: 2024
INFN-2023读取文件时发生错误: 2023
ZBRA-2023读取文件时发生错误: 2023
NVDA-2023读取文件时发生错误: 2023
ON-2023读取文件时发生错误: 2023
CSCO-2023读取文件时发生错误: 2023
SLAB-2023读取文件时发生错误: 2023
VSAT-2023读取文件时发生错误: 2023
PSTG-2023读取文件时发生错误: 2023
TDY-2023读取文件时发生错误: 2023
MTSI-2023读取文件时发生错误: 2023
NXPI-2023读取文件时发生错误: 2023
MCHP-2023读取文件时发生错误: 2023
IPGP-2023读取文件时发生错误: 2023
GLW-2023读取文件时发生错误: 2023
PSTG-2024

,CompanyTicker,Year,CSV.Length,ESGData,ESGLabel,BloombergOverall
0,ON,2016,1131,"[[0.0055219400674104, 0.0078070629388093, 0.00...","[N, G, N, N, G, N, N, N, N, N, N, N, N, N, N, ...",4.16
1,FFIV,2020,844,"[[0.004199112765491, 0.0238876864314079, 0.021...","[N, S, S, N, N, S, S, S, N, S, N, N, N, N, N, ...",2.70
2,NVDA,2016,451,"[[0.0117304930463433, 0.015321378596127, 0.010...","[N, N, N, N, S, N, N, N, N, N, N, N, N, N, N, ...",5.68
3,CSCO,2016,692,"[[0.0129457851871848, 0.0162092838436365, 0.01...","[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",4.53
4,SYNA,2018,697,"[[0.0032364279031753, 0.0065713287331163, 0.00...","[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",1.65
...,...,...,...,...,...,...
411,AMD,2017,1360,"[[0.0024161378387361, 0.001941418624483, 0.003...","[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",6.09
412,AGS,2018,259,"[[0.0055219400674104, 0.0078070629388093, 0.00...","[N, G, N, N, N, N, N, N, N, N, S, G, N, N, N, ...",1.23
413,VSH,2023,573,"[[0.0129457851871848, 0.0162092838436365, 0.01...","[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",0.00
414,SYNA,2015,380,"[[0.0129457851871848, 0.0162092838436365, 0.01...","[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",1.84


In [5]:
def ESGValueCount(labels):
    df = pd.DataFrame()
    df['Labels'] = labels
    return df['Labels'].value_counts()

In [6]:
for index,row in ESGOverallData.iterrows():
    ValueCount = ESGValueCount(row['ESGLabel'])
display(ESGOverallData)

,CompanyTicker,Year,CSV.Length,ESGData,ESGLabel,BloombergOverall
0,ON,2016,1131,"[[0.0055219400674104, 0.0078070629388093, 0.00...","[N, G, N, N, G, N, N, N, N, N, N, N, N, N, N, ...",4.16
1,FFIV,2020,844,"[[0.004199112765491, 0.0238876864314079, 0.021...","[N, S, S, N, N, S, S, S, N, S, N, N, N, N, N, ...",2.70
2,NVDA,2016,451,"[[0.0117304930463433, 0.015321378596127, 0.010...","[N, N, N, N, S, N, N, N, N, N, N, N, N, N, N, ...",5.68
3,CSCO,2016,692,"[[0.0129457851871848, 0.0162092838436365, 0.01...","[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",4.53
4,SYNA,2018,697,"[[0.0032364279031753, 0.0065713287331163, 0.00...","[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",1.65
...,...,...,...,...,...,...
411,AMD,2017,1360,"[[0.0024161378387361, 0.001941418624483, 0.003...","[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",6.09
412,AGS,2018,259,"[[0.0055219400674104, 0.0078070629388093, 0.00...","[N, G, N, N, N, N, N, N, N, N, S, G, N, N, N, ...",1.23
413,VSH,2023,573,"[[0.0129457851871848, 0.0162092838436365, 0.01...","[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",0.00
414,SYNA,2015,380,"[[0.0129457851871848, 0.0162092838436365, 0.01...","[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",1.84


In [29]:
NVDAData = ESGOverallData[ESGOverallData['CompanyTicker'] == "AAPL"]
# Sort by column: 'Year' (ascending)
NVDAData_sorted = NVDAData.sort_values(by='Year', ascending=True).reset_index(drop=True)
display(NVDAData_sorted)

,CompanyTicker,Year,CSV.Length,ESGData,ESGLabel,BloombergOverall
0,AAPL,2015,776,"[[0.0087282545864582, 0.0131738856434822, 0.00...","[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",3.73
1,AAPL,2016,777,"[[0.0087282545864582, 0.0131738856434822, 0.00...","[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",4.27
2,AAPL,2017,785,"[[0.0015240942593663, 0.0019455370493233, 0.00...","[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",4.27
3,AAPL,2018,773,"[[0.001494747120887, 0.0019286224851384, 0.002...","[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",4.41
4,AAPL,2019,548,"[[0.001426791655831, 0.0025409662630409, 0.002...","[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",5.13
5,AAPL,2020,553,"[[0.001426791655831, 0.0025409662630409, 0.002...","[N, N, N, N, N, N, N, N, N, N, N, N, N, E, N, ...",5.12
6,AAPL,2021,546,"[[0.001426791655831, 0.0025409662630409, 0.002...","[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",5.70
7,AAPL,2022,546,"[[0.001426791655831, 0.0025409662630409, 0.002...","[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",6.01
8,AAPL,2023,517,"[[0.001426791655831, 0.0025409662630409, 0.002...","[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",0.00


In [30]:
for index,row in NVDAData_sorted.iterrows():
    ValueCount = ESGValueCount(row['ESGLabel'])
    print(list(ValueCount.items()))


[('N', 762), ('S', 11), ('G', 3)]
[('N', 764), ('S', 11), ('G', 2)]
[('N', 770), ('S', 12), ('G', 3)]
[('N', 757), ('S', 10), ('G', 5), ('E', 1)]
[('N', 541), ('S', 3), ('G', 3), ('E', 1)]
[('N', 536), ('S', 11), ('G', 4), ('E', 2)]
[('N', 513), ('S', 28), ('G', 4), ('E', 1)]
[('N', 511), ('S', 24), ('E', 6), ('G', 5)]
[('N', 487), ('S', 21), ('E', 5), ('G', 4)]


In [28]:
NVDAData = pd.DataFrame([
    {'Year':2015,'N': 369, 'S': 37, 'E': 6, 'G': 5},
    {'Year':2016,'N': 375, 'S': 60, 'E': 10, 'G': 6},
    {'Year':2017,'N': 370, 'S': 74, 'E': 7, 'G': 9},
    {'Year':2018,'N': 381, 'S': 69, 'E': 7, 'G': 9},
    {'Year':2019,'N': 400, 'S': 70, 'E': 8, 'G': 9},
    {'Year':2020,'N': 484, 'S': 108, 'E': 7, 'G': 10},
    {'Year':2021,'N': 484, 'S': 104, 'E': 34, 'G': 17},
    {'Year':2022,'N': 564, 'S': 108, 'E': 32, 'G': 20},
    {'Year':2023,'N': 638, 'S': 128, 'E': 33, 'G': 32}
])
display(NVDAData)

,Year,N,S,E,G
0,2015,369,37,6,5
1,2016,375,60,10,6
2,2017,370,74,7,9
3,2018,381,69,7,9
4,2019,400,70,8,9
5,2020,484,108,7,10
6,2021,484,104,34,17
7,2022,564,108,32,20
8,2023,638,128,33,32
